In [51]:
!pip install folium

In [52]:
import folium
import pandas as pd
import numpy as np
import pyproj
import folium

# 2022년도 인허가난 카페숍들

In [53]:
data_open = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafe_visualization/restarants_ver.csv',
                   usecols=['사업장명','영업상태명','구', '인허가_년', '좌표정보(x)','좌표정보(y)']
                  )
data_open

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y)
0,폐업,종로구,2023,밀라커피 (Mila Coffee),201168.350187,453134.920223
1,폐업,강남구,2023,샌드랑,206279.206641,443733.802684
2,폐업,강남구,2023,빈스빈스경기도미술관점(세븐스애비뉴)(한시적),205130.591679,445590.096838
3,폐업,강남구,2023,커피크루(한시적),206279.206641,443733.802684
4,폐업,노원구,2023,위드고,205320.284767,461419.881795
...,...,...,...,...,...,...
54973,영업/정상,마포구,2017,굿애프터눈 합정점,192142.271363,449868.964586
54974,영업/정상,마포구,2017,보노보나마포합정점,192142.271363,449868.964586
54975,영업/정상,마포구,2017,셀렉토커피 상암누리꿈스퀘어점,190250.875092,453017.867203
54976,영업/정상,마포구,2016,로벤,192882.211403,449911.392128


In [54]:
#  2022년도이고, 영업/정상인 카페숍
data_open = data_open.loc[(data_open.인허가_년 == 2022) & (data_open.영업상태명 == '영업/정상')]
data_open

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y)
37735,영업/정상,서대문구,2022,카페인중독 신촌점,194170.408173,450453.360532
37738,영업/정상,은평구,2022,텐퍼센트 은평구청점,193673.435866,455497.602144
37825,영업/정상,강동구,2022,그릭이야(Greekiya),211885.742048,449345.290469
37865,영업/정상,강남구,2022,오브디저트(aube 디저트),204534.863512,445421.049700
37889,영업/정상,동작구,2022,찰리 앤 몰리,197465.772699,442084.788785
...,...,...,...,...,...,...
54660,영업/정상,동작구,2022,바모스(Vamos),192154.584656,442850.029125
54661,영업/정상,광진구,2022,파인드유어페이보릿,207527.646252,447640.296859
54662,영업/정상,강서구,2022,움라우트 카페,185576.686938,449074.180143
54663,영업/정상,서초구,2022,양재테니스장 카페,203093.373058,440988.316144


# 2022년도 폐업한 카페숍들

In [59]:
# 문자열이 들어가 있는 경우가 있다. 숫자형으로 데이터 타입을 변환한 후 이상값들은 제거

data_open['좌표정보(x)'] = pd.to_numeric(data_open['좌표정보(x)'], errors="coerce")
data_open['좌표정보(y)'] = pd.to_numeric(data_open['좌표정보(y)'], errors="coerce")

# data_location = data.dropna()
data_open.index=range(len(data_open))
data_open.tail()

<ipython-input-59-032f852b5e20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_open['좌표정보(x)'] = pd.to_numeric(data_open['좌표정보(x)'], errors="coerce")
<ipython-input-59-032f852b5e20>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_open['좌표정보(y)'] = pd.to_numeric(data_open['좌표정보(y)'], errors="coerce")


,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y)
2093,영업/정상,동작구,2022,바모스(Vamos),192154.584656,442850.029125
2094,영업/정상,광진구,2022,파인드유어페이보릿,207527.646252,447640.296859
2095,영업/정상,강서구,2022,움라우트 카페,185576.686938,449074.180143
2096,영업/정상,서초구,2022,양재테니스장 카페,203093.373058,440988.316144
2097,영업/정상,중랑구,2022,매머드익스프레스 면목시장점,207323.015721,452715.123718


In [60]:
# EPSG:2097에서 EPSG:4326로 변환하기
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]


In [61]:
# DataFrame -> NumPy Array 변환
coord = np.array(data_open.iloc[:,-2:])
coord

array([[194170.40817264, 450453.36053206],
       [193673.43586598, 455497.60214443],
       [211885.74204759, 449345.29046884],
       ...,
       [185576.68693757, 449074.18014279],
       [203093.3730577 , 440988.31614394],
       [207323.01572149, 452715.12371779]])

In [62]:
# 좌표계 정보 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-60-3275f79941a2>:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coo

array([[126.93192228,  37.55633388],
       [126.92625389,  37.60177847],
       [127.13239543,  37.54629335],
       ...,
       [126.83469138,  37.54381264],
       [127.03286787,  37.47106803],
       [127.08079767,  37.57670236]])

In [63]:
# 경도 위도 좌표계로 변환된 NumPy Array를 DataFrame에 추가
data_open['경도(x)'] = result[:, 0]
data_open['위도(y)'] = result[:, 1]

<ipython-input-63-4ced366fb124>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_open['경도(x)'] = result[:, 0]
<ipython-input-63-4ced366fb124>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_open['위도(y)'] = result[:, 1]


In [64]:
data_open.tail()

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y),경도(x),위도(y)
2093,영업/정상,동작구,2022,바모스(Vamos),192154.584656,442850.029125,126.909190,37.487813
2094,영업/정상,광진구,2022,파인드유어페이보릿,207527.646252,447640.296859,127.083063,37.530977
2095,영업/정상,강서구,2022,움라우트 카페,185576.686938,449074.180143,126.834691,37.543813
2096,영업/정상,서초구,2022,양재테니스장 카페,203093.373058,440988.316144,127.032868,37.471068
2097,영업/정상,중랑구,2022,매머드익스프레스 면목시장점,207323.015721,452715.123718,127.080798,37.576702


In [67]:
data_open = data_open.dropna()
data_open

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y),경도(x),위도(y)
0,영업/정상,서대문구,2022,카페인중독 신촌점,194170.408173,450453.360532,126.931922,37.556334
1,영업/정상,은평구,2022,텐퍼센트 은평구청점,193673.435866,455497.602144,126.926254,37.601778
2,영업/정상,강동구,2022,그릭이야(Greekiya),211885.742048,449345.290469,127.132395,37.546293
3,영업/정상,강남구,2022,오브디저트(aube 디저트),204534.863512,445421.049700,127.049190,37.511001
4,영업/정상,동작구,2022,찰리 앤 몰리,197465.772699,442084.788785,126.969247,37.480949
...,...,...,...,...,...,...,...,...
2093,영업/정상,동작구,2022,바모스(Vamos),192154.584656,442850.029125,126.909190,37.487813
2094,영업/정상,광진구,2022,파인드유어페이보릿,207527.646252,447640.296859,127.083063,37.530977
2095,영업/정상,강서구,2022,움라우트 카페,185576.686938,449074.180143,126.834691,37.543813
2096,영업/정상,서초구,2022,양재테니스장 카페,203093.373058,440988.316144,127.032868,37.471068


# map 시각화


In [73]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [75]:
from folium.plugins import MarkerCluster
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12,
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(data_open['위도(y)'], data_open['경도(x)']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
m